In [ ]:
import gtdynamics as gtd
from gtdynamics import ContactGoal, PointOnLink, Slice, Interval
from gtsam import Pose3, Point3

In [ ]:
# Load the vision 60 quadruped by Ghost robotics: https://youtu.be/wrBNJKZKg10
robot = gtd.CreateRobotFromFile(gtd.URDF_PATH + "/vision60.urdf");

In [ ]:
# feet
contact_in_com =(0.14, 0, 0)
LH = PointOnLink(robot.link("lower1"), contact_in_com)
LF = PointOnLink(robot.link("lower0"), contact_in_com)
RF = PointOnLink(robot.link("lower2"), contact_in_com)
RH = PointOnLink(robot.link("lower3"), contact_in_com)

# establish contact/goal pairs
contact_goals = [
    ContactGoal(LH, [-0.4, 0.16, 0]),
    ContactGoal(LF, [0.3, 0.16, 0]),
    ContactGoal(RF, [0.3, -0.16, 0]),
    ContactGoal(RH, [-0.4, -0.16, 0])
    ]
print(contact_goals)

In [ ]:
parameters = gtd.KinematicsParameters()
# parameters.method = gtd.KinematicsParameters.Method.SOFT_CONSTRAINTS # not yet wrapped
kinematics = gtd.Kinematics(parameters)
result = kinematics.inverse(Slice(4), robot, contact_goals)

In [ ]:
for goal in contact_goals:
    print(goal.link().name(), goal.satisfied(result,k=4,tol=1e-3))

In [ ]:
# interpolate

contact_goals2 = [
    ContactGoal(LH, [-0.4, 0.16, 0]),
    ContactGoal(LF, [0.3, 0.16, 0]),
    ContactGoal(RF, [0.3 + 0.10, -0.16, 0]),  # 10 cm on!
    ContactGoal(RH, [-0.4, -0.16, 0]),
]

k_start, k_end = 4, 8
interpolated_values = kinematics.interpolate(
    Interval(k_start, k_end), robot, contact_goals, contact_goals2
)
# interpolated_values.print("interpolated_values", gtd.GTDKeyFormatter)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for joint in robot.joints():
    joint_id = joint.id()
    ks = []
    angles = []
    for k in range(k_start, k_end + 1):
        key = gtd.JointAngleKey(joint_id, k)
        if interpolated_values.exists(key):
            ks.append(k)
            angles.append(interpolated_values.atDouble(key))

    if angles:
        fig.add_trace(go.Scatter(x=ks, y=angles, mode="lines+markers", name=joint.name()))

fig.update_layout(
    title="Interpolated Joint Angles",
    xaxis_title="k",
    yaxis_title="angle (rad)",
    template="plotly_white",
)
fig.show()

In [ ]:
for k in range(k_start, k_end + 1):
    print(f"--- k={k} ---")
    for goal in contact_goals:
        print(goal.link().name(), goal.satisfied(interpolated_values,k=k,tol=1e-3))